In [4]:
import pandas as pd
from pulp import LpProblem, LpVariable, lpSum, LpMinimize, LpAffineExpression, \
LpConstraint, PULP_CBC_CMD
import numpy as np
import pulp as plp
import seaborn as sns
from functions import *

In [5]:
foods = pd.read_csv('./02_Data_formatted/nv_df.csv', index_col='Unnamed: 0')
foods.index = [food.replace('-','_') for food in foods.index]
constraints = pd.read_csv('./02_Data_formatted/constraints.csv', index_col = 
                          'Unnamed: 0')

FileNotFoundError: [Errno 2] No such file or directory: './02_Data_formatted/nv_df.csv'

In [14]:
constraints_lp = constraints.drop(index = ['fat_total','carbohydrates','protein',
                                          'calories','cholesterol','chloride']).copy()

In [15]:
constraints = constraints_lp
# Adjust amino acid requirements for bodyweight
usr_mass = input("How many kilos do you weigh?")
usr_mass = int(usr_mass)
constraints.iloc[-11:,0] *= usr_mass

How many kilos do you weigh? 76


In [16]:
spices = foods[foods.calories<=15]
spices

,calories,protein,fat_total,saturated_fat,cholesterol,sodium,choline,folate,niacin,pantothenic_acid,...,biotin,chromium,cysteine_methionine,histidine,leucine,lysine,methionine,phenylalanine_tyrosine,threonine,valine
sea_vegetables_dulse_dried,10.85,1.81,0.09,0.02,0.0,14.96,0.00,45.60,0.68,0.16,...,0.00,0.00,0.08,0.04,0.16,0.07,0.05,0.17,0.07,0.13
lemons_and_limes_fresh_juice,13.42,0.21,0.15,0.02,0.0,0.61,3.11,12.20,0.06,0.08,...,0.18,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
soy_sauce_tamari,10.80,1.89,0.02,0.00,0.0,1005.48,6.91,3.24,1.25,0.07,...,0.00,0.00,0.05,0.04,0.13,0.13,0.03,0.16,0.07,0.09
basil_chopped_fresh,4.88,0.67,0.14,0.01,0.0,0.85,2.42,14.42,0.33,0.04,...,0.00,0.00,0.02,0.01,0.04,0.02,0.01,0.05,0.02,0.03
black_pepper_whole_peppercorns,14.56,0.60,0.19,0.08,0.0,1.16,0.66,0.99,0.12,0.08,...,0.00,0.93,0.02,0.01,0.06,0.01,0.01,0.06,0.01,0.03
cilantro_leaf_fresh,1.84,0.17,0.04,0.00,0.0,3.68,1.02,4.96,0.09,0.05,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
cinnamon_ground,12.84,0.21,0.06,0.02,0.0,0.52,0.57,0.31,0.11,0.02,...,0.00,0.00,0.00,0.01,0.01,0.01,0.00,0.02,0.01,0.01
cloves_ground,11.51,0.25,0.55,0.17,0.0,11.63,1.57,1.05,0.09,0.02,...,0.00,0.00,0.00,0.01,0.02,0.02,0.00,0.02,0.01,0.01
dill_sprig_fresh,1.91,0.15,0.05,0.00,0.0,2.71,0.00,6.67,0.08,0.02,...,0.00,0.00,0.00,0.00,0.01,0.01,0.00,0.00,0.00,0.01
ginger_slices_fresh,4.80,0.11,0.05,0.01,0.0,0.78,1.73,0.66,0.06,0.01,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [17]:
foods.sort_values(by='calories')[foods.sort_values(by='calories').calories<30]

,calories,protein,fat_total,saturated_fat,cholesterol,sodium,choline,folate,niacin,pantothenic_acid,...,biotin,chromium,cysteine_methionine,histidine,leucine,lysine,methionine,phenylalanine_tyrosine,threonine,valine
cilantro_leaf_fresh,1.84,0.17,0.04,0.00,0.0,3.68,1.02,4.96,0.09,0.05,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
dill_sprig_fresh,1.91,0.15,0.05,0.00,0.0,2.71,0.00,6.67,0.08,0.02,...,0.00,0.00,0.00,0.00,0.01,0.01,0.00,0.00,0.00,0.01
sage_dried,4.41,0.15,0.18,0.10,0.0,0.15,0.61,3.84,0.08,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ginger_slices_fresh,4.80,0.11,0.05,0.01,0.0,0.78,1.73,0.66,0.06,0.01,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
thyme_fresh,4.85,0.27,0.08,0.02,0.0,0.43,0.00,2.16,0.18,0.02,...,0.00,0.00,0.00,0.00,0.01,0.01,0.00,0.00,0.01,0.01
basil_chopped_fresh,4.88,0.67,0.14,0.01,0.0,0.85,2.42,14.42,0.33,0.04,...,0.00,0.00,0.02,0.01,0.04,0.02,0.01,0.05,0.02,0.03
oregano_leaf_dried,5.30,0.18,0.09,0.03,0.0,0.50,0.65,4.74,0.16,0.02,...,0.00,0.00,0.00,0.00,0.01,0.01,0.00,0.02,0.01,0.01
peppermint_leaves_fresh,5.32,0.28,0.07,0.02,0.0,2.36,0.00,8.66,0.20,0.03,...,0.00,0.00,0.00,0.01,0.02,0.01,0.00,0.02,0.01,0.01
rosemary_fresh,5.50,0.14,0.25,0.12,0.0,1.09,0.00,4.58,0.07,0.03,...,0.00,0.00,0.00,0.00,0.01,0.01,0.00,0.01,0.01,0.01
soy_sauce_tamari,10.80,1.89,0.02,0.00,0.0,1005.48,6.91,3.24,1.25,0.07,...,0.00,0.00,0.05,0.04,0.13,0.13,0.03,0.16,0.07,0.09


In [ ]:
def SeriesCutoff(correlations):
    
    
    

In [2]:
foods.corr()[foods.corr()>.5]['phenylalanine_tyrosine']

NameError: name 'foods' is not defined

In [18]:
# categorize constraints for exclusions from mathematicsl model
amino_acids = SeriesCutoff(
    foods.corr()[foods.corr()>.5]['phenylalanine_tyrosine'],
            .90).index.values
spices.index
foods = foods.loc[[food for food in foods.index.values if 
                   food not in spices.index],:]

NameError: name 'SeriesCutoff' is not defined

In [ ]:
# Rescale constraints to match units in foods
rescale = ['chromium','copper','cysteine_methionine', 'histidine', 'leucine', 'lysine',
       'methionine', 'phenylalanine_tyrosine', 'threonine', 'valine']
constraints.loc[rescale,:] = constraints.loc[rescale,:]/1000
constraints.loc['potassium',:] *= 1000

In [ ]:
# Adjust constraint values
constraints.loc['fat_total','min']=0
constraints.loc['protein','min']=0
constraints.loc['chloride','max']=3100
constraints.loc['biotin','min'] = 40
constraints.loc['sodium','min'] = 0

In [ ]:
lp_formulation = formulateMP(foods,constraints)

In [ ]:
lp_formulation.solve()

In [ ]:
f_i = infeasibilitySearch(foods,constraints)

###### What do the loadings look like for the solution with the infeasible constraints removed?

In [ ]:
i_c = f_i[1]
feasible = f_i[0]
f_c = constraints.iloc[feasible]

In [ ]:
model = formulateMP(foods,f_c)

In [ ]:
plp.listSolvers(onlyAvailable=True)

In [ ]:
model.solve(solver=PULP_CBC_CMD(msg=False))

In [ ]:
total_cals = 0
food_servings = {}
for v in model.variables():
    total_cals += v.varValue*foods.at[v.name,'calories']
    food_servings[v.name] = v.varValue
food_servings[v.name] = v.varValue
food_srv_dict = {food:srv for food, srv in food_servings.items() if srv != 0}

In [ ]:
food_srv_dict

###### Investigating failed constraints

In [ ]:
nutrient_total = pd.Series(data = [0]*foods.shape[1], index = foods.columns)
for food, srv in food_srv_dict.items():
    foods.loc[food,:]
    nutrient_total += foods.loc[food,:]*srv
nutrient_total

In [ ]:
constraints['min']

In [ ]:
nutrient_total/constraints['min']

###### Plot contribution breakdown by food type for each nutrient

In [ ]:
pd.DataFrame.from_dict({food:foods.at[food,'protein']*srv for food, srv in food_srv_dict.items()},orient ='index',columns=['grams_protein'] )

In [ ]:
constraints

In [ ]:
foods['calcium']

In [ ]:
food_srv_dict

why so much calcium

In [ ]:
def makePlot(fd_srv_dict, nutrient,foods):
'''make a plot to show the nutrient contribution for each of the foods'''
pd.DataFrame.from_dict(
    {food:foods.at[food,'protein']*srv for food, srv in food_srv_dict.items()},
                        orient ='index',
                        columns=['grams_protein'] )

In [ ]:
for nutrient in constraints.index:
    pd.DataFrame.from_dict({food:})
    sns.barplot()